In [2]:
import cv2
from PIL import Image, ImageOps
import argparse
import os
import glob
import matplotlib.pyplot as plt
import multiprocessing as mp
from joblib import Parallel, delayed
import pydicom
import time
import numpy as np
import torch
import random
import pandas as pd
from pydicom.pixel_data_handlers import apply_windowing
import imageio
import time
import shutil

In [2]:
meta = pd.read_csv('D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\meta_new.csv', index_col='Unnamed: 0')
meta

,patient_id,image_id,view,laterality,cancer
0,10006,1874946579,CC,R,0
1,10006,1864590858,MLO,R,0
2,10006,462822612,CC,L,0
3,10006,1459541791,MLO,L,0
4,10011,541722628,CC,R,0
...,...,...,...,...,...
53745,D2-0747,D2-0747-1-3,MLO,R,1
53746,D2-0748,D2-0748-1-4,CC,R,1
53747,D2-0748,D2-0748-1-3,MLO,R,1
53748,D2-0749,D2-0749-1-2,CC,L,1


In [3]:
meta.cancer.value_counts()

0    49490
1     4260
Name: cancer, dtype: int64

In [4]:
# Remove vindr
vindr = pd.read_csv('D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\Vindr.csv', index_col='Unnamed: 0')

vindr

,patient_id,image_id,view,laterality,cancer
0,10200894bf6f74b2ae92a0cecd37c60c,581b0e4d471a5c0adc6888cb038fa722,CC,R,1
1,10200894bf6f74b2ae92a0cecd37c60c,29896919b15b2dd978a36b2f68a5ad93,MLO,R,1
2,10200894bf6f74b2ae92a0cecd37c60c,8992f69db04084cbb4b3b31f6735e4eb,CC,L,0
3,10200894bf6f74b2ae92a0cecd37c60c,af0f6ad067fdf3e94c630ad4f5e5151f,MLO,L,0
4,450f7f9b2aa32754b076f0f53f0d8efc,33be94e3d81065693202c21aa662cfd2,CC,R,0
...,...,...,...,...,...
447,8ae2db86499819584e175cadf9099d25,8ec34cef1eb98d7c121509c8342b2212,MLO,L,0
448,3bdcc5d8c49f1e2b6332dc0a039ca35d,10e0f362333df810ac84a9db8fb3fd42,CC,R,1
449,3bdcc5d8c49f1e2b6332dc0a039ca35d,85a6579cbdc403cfc4dde0a8149ed855,MLO,R,1
450,3bdcc5d8c49f1e2b6332dc0a039ca35d,3cd51ee99070c4d625d52b848d5e9bfc,CC,L,0


In [5]:
meta = meta[~meta.image_id.isin(vindr.image_id)]
meta

,patient_id,image_id,view,laterality,cancer
0,10006,1874946579,CC,R,0
1,10006,1864590858,MLO,R,0
2,10006,462822612,CC,L,0
3,10006,1459541791,MLO,L,0
4,10011,541722628,CC,R,0
...,...,...,...,...,...
53745,D2-0747,D2-0747-1-3,MLO,R,1
53746,D2-0748,D2-0748-1-4,CC,R,1
53747,D2-0748,D2-0748-1-3,MLO,R,1
53748,D2-0749,D2-0749-1-2,CC,L,1


In [6]:
meta.cancer

0        0
1        0
2        0
3        0
4        0
        ..
53745    1
53746    1
53747    1
53748    1
53749    1
Name: cancer, Length: 53298, dtype: int64

In [87]:
import numpy as np
import pandas as pd
import multiprocessing as mp
class Setting_data:
    def __init__(self, data):
        
        self.data= data[['patient_id', 'image_id', 'view', 'laterality','cancer']]
        
        self.id = 0
        self.par = 0
        self.group = None
        
        
        self.meta_pro = pd.DataFrame(columns=['patient_id', 'image_id', 'view', 'laterality', 'cancer'])
    def set_par(self, par):
        self.par = par
    ## Zip columns img and cancer
    def zip_idimg_cancer(self):
        # Tạo id_img_zip để tận dụng cancer để sort xử lý ưu tiênc
        '''
        Chú ý dữ liệu phải được resetindex trước kh xử lý 
        '''
        self.data['id_img_zip'] = None
        for i in range(len(self.data)):
            # print(f'{self.data.cancer[i]}_{self.data.image_id[i]}')
            self.data.id_img_zip[i] = f'{self.data.cancer[i]}_{self.data.image_id[i]}'
        # ## Nhóm dữ liệu lại để xử lý
        self.group = self.data.groupby(['patient_id', 'view', 'laterality', 'id_img_zip']).size()
        self.data = self.data[(self.data.view == 'CC') |(self.data.view == 'MLO')]
        
    
    # Hàm filter_for2x là hàm dùng để cân bằng dữ liệu
    def filter_for_2x(self, par, lat):
        
        A = list(self.group[par]['CC'][lat].index)
        B = list(self.group[par]['MLO'][lat].index)
        
        A = sorted(A, key=lambda x: x[0],  reverse=True)# ls[0][0]
        B = sorted(B, key=lambda x: x[0],  reverse=True)# ls[0][0]
        i = 0
        j = 0
        # print(A)
        # print(B)
        while i < len(A) and j < len(B):
            a = A[i]
            b = B[i]
            # print(f"Lay a: {a.split('_')} \n lay b:{a.split('_')}")
            cancer_a, a = a.split('_')
            cancer_b, b = b.split('_')
            # print(a, b)
            ## update data
            # return
            
            self.meta_pro.loc[self.id] = [par, a, 'CC', lat, int(cancer_a)]
            # return
            self.meta_pro.loc[self.id+1] = [par, b, 'MLO', lat, int(cancer_b)]
            
            self.id+=2
            i+=1
            j+=1
  
    def check_cancer(self, par, views, lat):
        try: 
            return len(self.group[par][views][lat].index) >0
        except:
            return False
    def filter_main(self, par, lat):
        # Kiểm tra nếu CC + lat và MLO + lat mà đều chứa hình ảnh true
        if self.check_cancer(par, 'CC', lat) and self.check_cancer(par, 'MLO', lat):
           self.filter_for_2x(par, lat=lat)

            
    def save_file(self, path):
        self.meta_pro.to_csv(path)
    def procee_par(self, par):
        ## Process img R
    
        self.filter_main(par, lat='R')

        ## Process img L
        self.filter_main(par, lat='L')
    def process_multi(self, par):
        # print(par)
        self.procee_par(par)
    def multi_process(self, list_par):
        self.zip_idimg_cancer()
        for i in list_par:
            self.procee_par(i)

    

Cancer

In [88]:
ddsm = pd.read_csv('D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\DDSM.csv',index_col='Unnamed: 0')
ddsm

,patient_id,image_id,view,laterality,cancer
0,0001DDSM,0,CC,L,1
1,0001DDSM,1,MLO,L,1
2,0001DDSM,2,CC,R,1
3,0001DDSM,3,MLO,R,1
4,0002DDSM,4,CC,L,1
...,...,...,...,...,...
5119,4607DDSM,5119,MLO,R,0
5120,4608DDSM,5120,CC,L,0
5121,4608DDSM,5121,MLO,L,0
5122,4608DDSM,5122,CC,R,0


In [89]:
from Evaluate import *
from Setting_data import *

In [90]:
ev = Evaluate(ddsm, 'ddsm')
ev.show()

Dataset ddsm 
 Số lượng image: 5124 
 Số luong patient: 1137
So luong CC và MLO
CC     2562
MLO    2562
Name: view, dtype: int64
So luong L và R
view  laterality
CC    L             1281
      R             1281
MLO   L             1281
      R             1281
dtype: int64
So luong img cancer: 2716


In [10]:
import numpy as np
import pandas as pd
import multiprocessing as mp
class Setting_data:
    def __init__(self, data):
        
        self.data= data[['patient_id', 'image_id', 'view', 'laterality','cancer']]
        
        self.id = 0
        self.par = 0
        self.group = None
        
        
        self.meta_pro = pd.DataFrame(columns=['patient_id', 'image_id', 'view', 'laterality', 'cancer'])
    def set_par(self, par):
        self.par = par
    ## Zip columns img and cancer
    def zip_idimg_cancer(self):
        # Tạo id_img_zip để tận dụng cancer để sort xử lý ưu tiênc
        '''
        Chú ý dữ liệu phải được resetindex trước kh xử lý 
        '''
        self.data['id_img_zip'] = None
        for i in range(len(self.data)):
            # print(f'{self.data.cancer[i]}_{self.data.image_id[i]}')
            self.data.id_img_zip[i] = f'{self.data.cancer[i]}_{self.data.image_id[i]}'
        # ## Nhóm dữ liệu lại để xử lý
        self.group = self.data.groupby(['patient_id', 'view', 'laterality', 'id_img_zip']).size()
        self.data = self.data[(self.data.view == 'CC') |(self.data.view == 'MLO')]
        
    
    # Hàm filter_for2x là hàm dùng để cân bằng dữ liệu
    def filter_for_2x(self, par, lat):
        
        A = list(self.group[par]['CC'][lat].index)
        B = list(self.group[par]['MLO'][lat].index)
        
        A = sorted(A, key=lambda x: x[0],  reverse=True)# ls[0][0]
        B = sorted(B, key=lambda x: x[0],  reverse=True)# ls[0][0]
        i = 0
        j = 0
        # print(A)
        # print(B)
        while i < len(A) and j < len(B):
            a = A[i]
            b = B[i]
            # print(f"Lay a: {a.split('_')} \n lay b:{a.split('_')}")
            cancer_a, a = a.split('_')
            cancer_b, b = b.split('_')
            # print(a, b)
            ## update data
            # return
            
            self.meta_pro.loc[self.id] = [par, a, 'CC', lat, int(cancer_a)]
            # return
            self.meta_pro.loc[self.id+1] = [par, b, 'MLO', lat, int(cancer_b)]
            
            self.id+=2
            i+=1
            j+=1
  
    def check_cancer(self, par, views, lat):
        try: 
            return len(self.group[par][views][lat].index) >0
        except:
            return False
    def filter_main(self, par, lat):
        # Kiểm tra nếu CC + lat và MLO + lat mà đều chứa hình ảnh true
        if self.check_cancer(par, 'CC', lat) and self.check_cancer(par, 'MLO', lat):
           self.filter_for_2x(par, lat=lat)

            
    def save_file(self, path):
        self.meta_pro.to_csv(path)
    def procee_par(self, par):
        ## Process img R
    
        self.filter_main(par, lat='R')

        ## Process img L
        self.filter_main(par, lat='L')
    def process_multi(self, par):
        # print(par)
        self.procee_par(par)
    def multi_process(self, list_par):
        self.zip_idimg_cancer()
        for i in list_par:
            self.procee_par(i)

    

In [11]:
cmmd_main.image_id = cmmd_main.image_id.apply(lambda x: x.replace('%=', '_'))

In [12]:
cmmd_main

,Unnamed: 0,index,patient_id,image_id,laterality,cancer,path,view
0,0,0,D1-0001,D1-0001-1-1,R,0,D:/OneDrive - Industrial University of HoChiMi...,MLO
1,1,1,D1-0001,D1-0001-1-2,R,0,D:/OneDrive - Industrial University of HoChiMi...,CC
2,2,2,D1-0002,D1-0002-1-1,L,0,D:/OneDrive - Industrial University of HoChiMi...,MLO
3,3,3,D1-0002,D1-0002-1-2,L,0,D:/OneDrive - Industrial University of HoChiMi...,CC
4,4,4,D1-0003,D1-0003-1-1,L,0,D:/OneDrive - Industrial University of HoChiMi...,MLO
...,...,...,...,...,...,...,...,...
3736,3736,3736,D2-0747,D2-0747-1-4,R,1,D:/OneDrive - Industrial University of HoChiMi...,CC
3737,3737,3737,D2-0748,D2-0748-1-3,R,1,D:/OneDrive - Industrial University of HoChiMi...,MLO
3738,3738,3738,D2-0748,D2-0748-1-4,R,1,D:/OneDrive - Industrial University of HoChiMi...,CC
3739,3739,3739,D2-0749,D2-0749-1-1,L,1,D:/OneDrive - Industrial University of HoChiMi...,MLO


In [13]:
st = Setting_data(cmmd_main)
st.multi_process(cmmd_main.patient_id.unique())

C:\Users\nguye\AppData\Local\Temp\ipykernel_6264\226691407.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['id_img_zip'] = None
C:\Users\nguye\AppData\Local\Temp\ipykernel_6264\226691407.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.id_img_zip[i] = f'{self.data.cancer[i]}_{self.data.image_id[i]}'


In [282]:
st.meta_pro.to_csv('D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\DATA_STANDARD\CMMD.csv')

In [273]:
ls = glob.glob('D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\Dataset_VIEW\D1-0001\*')

In [271]:
ls

['D:\\OneDrive - Industrial University of HoChiMinh City\\WORKBASE\\Project-rsna-breast-cancer-detection\\Data_main\\Dataset_VIEW\\0001DDSM',
 'D:\\OneDrive - Industrial University of HoChiMinh City\\WORKBASE\\Project-rsna-breast-cancer-detection\\Data_main\\Dataset_VIEW\\0002DDSM',
 'D:\\OneDrive - Industrial University of HoChiMinh City\\WORKBASE\\Project-rsna-breast-cancer-detection\\Data_main\\Dataset_VIEW\\0003DDSM',
 'D:\\OneDrive - Industrial University of HoChiMinh City\\WORKBASE\\Project-rsna-breast-cancer-detection\\Data_main\\Dataset_VIEW\\0004DDSM',
 'D:\\OneDrive - Industrial University of HoChiMinh City\\WORKBASE\\Project-rsna-breast-cancer-detection\\Data_main\\Dataset_VIEW\\0005DDSM',
 'D:\\OneDrive - Industrial University of HoChiMinh City\\WORKBASE\\Project-rsna-breast-cancer-detection\\Data_main\\Dataset_VIEW\\0006DDSM',
 'D:\\OneDrive - Industrial University of HoChiMinh City\\WORKBASE\\Project-rsna-breast-cancer-detection\\Data_main\\Dataset_VIEW\\0007DDSM',
 'D:\\

In [14]:
st.meta_pro

,patient_id,image_id,view,laterality,cancer
0,D1-0001,D1-0001-1-2,CC,R,0
1,D1-0001,D1-0001-1-1,MLO,R,0
2,D1-0002,D1-0002-1-2,CC,L,0
3,D1-0002,D1-0002-1-1,MLO,L,0
4,D1-0003,D1-0003-1-2,CC,L,0
...,...,...,...,...,...
3733,D2-0747,D2-0747-1-3,MLO,R,1
3734,D2-0748,D2-0748-1-4,CC,R,1
3735,D2-0748,D2-0748-1-3,MLO,R,1
3736,D2-0749,D2-0749-1-2,CC,L,1


In [15]:
ev = Evaluate(st.meta_pro, 'DDSM')
ev.show()

Dataset DDSM 
 Số lượng image: 3738 
 Số luong patient: 1772
So luong CC và MLO
CC     1869
MLO    1869
Name: view, dtype: int64
So luong L và R
view  laterality
CC    L             987
      R             882
MLO   L             987
      R             882
dtype: int64
So luong img cancer: 2632


In [36]:
st.meta_pro.cancer.value_counts()

1    2632
0    1106
Name: cancer, dtype: int64

In [91]:
meta = ddsm
meta

,patient_id,image_id,view,laterality,cancer
0,0001DDSM,0,CC,L,1
1,0001DDSM,1,MLO,L,1
2,0001DDSM,2,CC,R,1
3,0001DDSM,3,MLO,R,1
4,0002DDSM,4,CC,L,1
...,...,...,...,...,...
5119,4607DDSM,5119,MLO,R,0
5120,4608DDSM,5120,CC,L,0
5121,4608DDSM,5121,MLO,L,0
5122,4608DDSM,5122,CC,R,0


In [92]:

cancer = meta[meta.cancer == 1].reset_index().drop('index', axis=1)
cancer

,patient_id,image_id,view,laterality,cancer
0,0001DDSM,0,CC,L,1
1,0001DDSM,1,MLO,L,1
2,0001DDSM,2,CC,R,1
3,0001DDSM,3,MLO,R,1
4,0002DDSM,4,CC,L,1
...,...,...,...,...,...
2711,4193DDSM,2711,MLO,R,1
2712,4199DDSM,2712,CC,L,1
2713,4199DDSM,2713,MLO,L,1
2714,4199DDSM,2714,CC,R,1


In [93]:
s = 0;
cnt = 0

In [94]:
for i in cancer.groupby('patient_id')['image_id'].count():
    if s > 2000:
        break
    cnt+=1
    s+=i

In [95]:
print(s, cnt)

2004 501


In [96]:
# con 712

In [97]:
len(cancer.groupby('patient_id')['image_id'].count())

679

In [98]:
cancer.groupby('patient_id')['image_id'].count().value_counts()

4    679
Name: image_id, dtype: int64

In [99]:
len(cancer.patient_id.unique())

679

In [100]:
s

2004

In [101]:
# Chúng ta sẽ lấy cỡ 1975 bệnh nhân cancer đầu làm train có 3802 image
# còn 133 bệnh nhân và khoảng  232 image cho val
# tập test sẽ là số bênh nhân trong DDSM

In [102]:
par = meta[meta.cancer == 1].patient_id.unique()
len(par)

679

In [103]:
cancer_train = cancer[cancer.patient_id.isin(par[:cnt])].reset_index().drop('index', axis=1)
cancer_train

,patient_id,image_id,view,laterality,cancer
0,0001DDSM,0,CC,L,1
1,0001DDSM,1,MLO,L,1
2,0001DDSM,2,CC,R,1
3,0001DDSM,3,MLO,R,1
4,0002DDSM,4,CC,L,1
...,...,...,...,...,...
1999,3078DDSM,1999,MLO,R,1
2000,3079DDSM,2000,CC,L,1
2001,3079DDSM,2001,MLO,L,1
2002,3079DDSM,2002,CC,R,1


In [104]:
int(0.2*( len(cancer) ))

543

In [105]:
cnt+int(0.2*len(cancer))

1044

In [106]:
cnt

501

In [111]:
500/4

125.0

In [112]:
cancer_val = cancer[cancer.patient_id.isin(par[cnt:cnt+125])].reset_index().drop('index', axis=1)
cancer_val

,patient_id,image_id,view,laterality,cancer
0,3080DDSM,2004,CC,L,1
1,3080DDSM,2005,MLO,L,1
2,3080DDSM,2006,CC,R,1
3,3080DDSM,2007,MLO,R,1
4,3081DDSM,2008,CC,L,1
...,...,...,...,...,...
495,4129DDSM,2499,MLO,R,1
496,4130DDSM,2500,CC,L,1
497,4130DDSM,2501,MLO,L,1
498,4130DDSM,2502,CC,R,1


In [113]:
cnt+226

727

In [114]:
ls = pd.concat([(cancer_train.image_id), (cancer_val.image_id)])
ls

0         0
1         1
2         2
3         3
4         4
       ... 
495    2499
496    2500
497    2501
498    2502
499    2503
Name: image_id, Length: 2504, dtype: int64

In [115]:
cancer_test = cancer[~cancer.image_id.isin(ls)].reset_index().drop('index', axis=1)
cancer_test

,patient_id,image_id,view,laterality,cancer
0,4131DDSM,2504,CC,L,1
1,4131DDSM,2505,MLO,L,1
2,4131DDSM,2506,CC,R,1
3,4131DDSM,2507,MLO,R,1
4,4132DDSM,2508,CC,L,1
...,...,...,...,...,...
207,4193DDSM,2711,MLO,R,1
208,4199DDSM,2712,CC,L,1
209,4199DDSM,2713,MLO,L,1
210,4199DDSM,2714,CC,R,1


In [41]:
# Danh gia lai so luong MLO va CC trong tap cancer

In [116]:
from Evaluate import *

In [117]:
evtrain = Evaluate(cancer_train, 'cancer_train')
evval = Evaluate(cancer_val, 'cancer_val')
evtest = Evaluate(cancer_test, 'cancer_val')

In [118]:
evtrain.show()


Dataset cancer_train 
 Số lượng image: 2004 
 Số luong patient: 501
So luong CC và MLO
CC     1002
MLO    1002
Name: view, dtype: int64
So luong L và R
view  laterality
CC    L             501
      R             501
MLO   L             501
      R             501
dtype: int64
So luong img cancer: 2004


In [119]:
evval.show()

Dataset cancer_val 
 Số lượng image: 500 
 Số luong patient: 125
So luong CC và MLO
CC     250
MLO    250
Name: view, dtype: int64
So luong L và R
view  laterality
CC    L             125
      R             125
MLO   L             125
      R             125
dtype: int64
So luong img cancer: 500


In [120]:
evtest.show()

Dataset cancer_val 
 Số lượng image: 212 
 Số luong patient: 53
So luong CC và MLO
CC     106
MLO    106
Name: view, dtype: int64
So luong L và R
view  laterality
CC    L             53
      R             53
MLO   L             53
      R             53
dtype: int64
So luong img cancer: 212


In [121]:

nocancer = meta[meta.cancer == 0].reset_index().drop('index', axis=1)
# nocancer.groupby(['patient_id', 'view', 'laterality']).
nocancer

,patient_id,image_id,view,laterality,cancer
0,0002DDSM,2716,CC,L,0
1,0002DDSM,2717,MLO,L,0
2,0002DDSM,2718,CC,R,0
3,0002DDSM,2719,MLO,R,0
4,0003DDSM,2720,CC,L,0
...,...,...,...,...,...
2403,4607DDSM,5119,MLO,R,0
2404,4608DDSM,5120,CC,L,0
2405,4608DDSM,5121,MLO,L,0
2406,4608DDSM,5122,CC,R,0


In [122]:
par = meta[meta.cancer == 0].patient_id.unique()

# nocancer_train = nocancer[nocancer.patient_id.isin(par[:422])].reset_index().drop('index', axis=1)
# nocancer_train

In [50]:
# nocancer_val = nocancer[nocancer.patient_id.isin(par[422:422+120])].reset_index().drop('index', axis=1)
# nocancer_val

In [51]:
# nocancer_test = nocancer[nocancer.patient_id.isin(par[422+120:])].reset_index().drop('index', axis=1)
# nocancer_test

In [52]:
# ev = Evaluate(nocancer, "NO")
# ev.show()

In [123]:
def processing_train_nocancer(data, rate_train, std_train, std_val):
    level = std_train * rate_train
    nocancer = data[data.cancer==0].reset_index()
    no_par =  nocancer.groupby(['patient_id'])['image_id'].count()
    cnt = 0
    s = 0
    for i in no_par:
        if s > level:
            break
        s+=i
        cnt+=1
    par = no_par.index
    nocancer_train = nocancer[nocancer.patient_id.isin(par[:cnt])].reset_index()
    
    ## Tap val
    s_val = 0
    cnt_val = 0
    for i in no_par[cnt: ]:
        if s_val > std_val:
            break
        s_val += i
        cnt_val+=1
    print(cnt, cnt_val)
    nocancer_val = nocancer[nocancer.patient_id.isin(par[cnt:cnt + cnt_val])].reset_index()
    return nocancer_train[["patient_id",	"image_id",	"view",	"laterality",	"cancer"]],nocancer_val[["patient_id",	"image_id",	"view",	"laterality",	"cancer"]]


In [124]:
class Evaluate:
    def __init__(self, meta_pro, name):
        self.al_data = meta_pro.reset_index()
        self.name = name
    def quatity_cc_mlo(self):
        return f"\nSo luong CC và MLO\n{self.al_data.view.value_counts()}\n"
        # print(self.al_data.view.value_counts())
    def lelf_right(self):
        return f"\nSo luong L và R\n{self.al_data.groupby(['view', 'laterality']).size()}\n"
        # print(self.al_data.groupby(['view', 'laterality']).size())
    def show_one(self, par):
        return self.al_data[self.al_data.patient_id == par]
    def quanlity_cancer(self):
        sl = len(self.al_data[self.al_data.cancer == 1])
        return f"So luong img cancer: {sl}"
        # print(self.al_data[self.al_data.cancer == 1])
    def show(self, path):
        ls = len(self.al_data)
        par = len(self.al_data.patient_id.unique())
        # print( 
        # self.quatity_cc_mlo()
        # self.lelf_right()
        # self.quanlity_cancer()
        f = open(path, 'a', encoding='utf-8')
        f.write(f'Dataset {self.name} \n Số lượng image: {ls} \n Số luong patient: {par} {self.quatity_cc_mlo()} {self.lelf_right()} {self.quanlity_cancer()}\n\n +++++++++++++++\n\n')
        f.close()

In [125]:
def save(cancer_train, nocancer_train, cancer_val, nocancer_val, cancer_test, nocancer_test, name = "Rong",  name_folder = "Rong"):
    path = f'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/CSV_MAIN/{name_folder}/{name}'
    os.makedirs(path)
    train = pd.concat([cancer_train, nocancer_train]).reset_index().drop('index', axis=1)
    val = pd.concat([cancer_val, nocancer_val]).reset_index().drop('index', axis=1)
    test = pd.concat([cancer_test, nocancer_test]).reset_index().drop('index', axis=1)
    # test = pd.read_csv(r'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection\DATA_STANDARD\DDSM.csv', index_col='Unnamed: 0')
    ev = Evaluate(train, "Train")
    ev1 = Evaluate(val, "Val")
    ev2 = Evaluate(test, "Test")
    ev.show(f'{path}/description.txt')
    ev1.show(f'{path}/description.txt')
    ev2.show(f'{path}/description.txt')
    train.to_csv(f'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/CSV_MAIN/{name_folder}/{name}/train.csv')
    val.to_csv(f'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/CSV_MAIN/{name_folder}/{name}/val.csv')
    test.to_csv(f'D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/CSV_MAIN/{name_folder}/{name}/test.csv')

In [126]:
nocancer

,patient_id,image_id,view,laterality,cancer
0,0002DDSM,2716,CC,L,0
1,0002DDSM,2717,MLO,L,0
2,0002DDSM,2718,CC,R,0
3,0002DDSM,2719,MLO,R,0
4,0003DDSM,2720,CC,L,0
...,...,...,...,...,...
2403,4607DDSM,5119,MLO,R,0
2404,4608DDSM,5120,CC,L,0
2405,4608DDSM,5121,MLO,L,0
2406,4608DDSM,5122,CC,R,0


In [127]:
def process_rate(level, stdtest = 0):
    nocancer_train, nocancer_val = processing_train_nocancer(meta, level, 2302, 500)
    nocancer_test = nocancer[(~nocancer.patient_id.isin(nocancer_val.patient_id)) & (~nocancer.patient_id.isin(nocancer_train.patient_id))]
    nocancer_test =nocancer_test[:stdtest]
    # save(cancer_train, nocancer_train, cancer_val, nocancer_val, f'Rate1{level}')
    save(cancer_train, nocancer_train, cancer_val, nocancer_val, cancer_test, nocancer_test,f'rate1{level}', 'CMMD')
    if len(nocancer_val[nocancer_val.image_id.isin(nocancer_train.image_id)]) == 0 :
        print(len(nocancer_train), len(nocancer_val))
        print("GOOD")
    else:
        print("BAD")

In [128]:
len(cancer_test),len(cancer_val), len(cancer_train)

(212, 500, 2004)

In [63]:
1106 - 110 - 221

775

In [64]:
110, 221, 775

(110, 221, 775)

In [129]:
from sklearn.model_selection import train_test_split


In [130]:
no_cancer_train, no_cancer_test_val = train_test_split(nocancer, test_size=0.3, random_state=42)
no_cancer_val, no_cancer_test = train_test_split(no_cancer_test_val,  test_size=0.3, random_state=42)

In [131]:
no_cancer_val, no_cancer_test = train_test_split(no_cancer_test_val,  test_size=0.3, random_state=42)

In [132]:
len(no_cancer_train), len(no_cancer_val), len(no_cancer_test)

(1685, 506, 217)

In [134]:
save(cancer_train, no_cancer_train, cancer_val, no_cancer_val, cancer_test, no_cancer_test,'rate11', 'DDSM')

In [252]:
len(cancer_val)

454

In [253]:
len(cancer_test)

176

In [254]:
process_rate(1, 200)
process_rate(2, 200)
process_rate(3, 200)

492 0
1106 0
GOOD
492 0
1106 0
GOOD
492 0
1106 0
GOOD


In [117]:
process_rate(3)

2949 72
12002 302
GOOD


In [118]:
process_rate(4)

3923 78
16004 302
GOOD


In [120]:
## Move image to folder cancer 

In [2]:
data = pd.read_csv("D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\meta_data.csv", index_col='Unnamed: 0')


In [3]:
data

,patient_id,image_id,view,laterality,cancer
0,10006,1874946579,CC,R,0
1,10006,1864590858,MLO,R,0
2,10006,462822612,CC,L,0
3,10006,1459541791,MLO,L,0
4,10011,541722628,CC,R,0
...,...,...,...,...,...
53745,D2-0747,D2-0747-1-3,MLO,R,1
53746,D2-0748,D2-0748-1-4,CC,R,1
53747,D2-0748,D2-0748-1-3,MLO,R,1
53748,D2-0749,D2-0749-1-2,CC,L,1


In [4]:
cancer  = data[data.cancer == 1]
nocancer  = data[data.cancer == 0]

In [5]:
# patient -> view -> Lat -> view_lat_id_image

In [6]:
meta = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\meta_data.csv')

In [7]:
train = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\Rate11\train.csv')
val = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\Rate11\val.csv')
test = pd.read_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\Rate11\test.csv')

In [8]:
train.cancer.value_counts()

0    4002
1    3802
Name: cancer, dtype: int64

In [9]:
dict_cancer = {}

for i in range(len(meta)):
    dict_cancer[meta.image_id[i]] = meta.cancer[i]

In [10]:
for i in range(len(train)):
    train.cancer[i] = train.cancer[i] and dict_cancer[train.image_id[i]]

C:\Users\nguye\AppData\Local\Temp\ipykernel_18656\2621381458.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.cancer[i] = train.cancer[i] and dict_cancer[train.image_id[i]]


In [11]:
for i in range(len(val)):
    val.cancer[i] = val.cancer[i] and dict_cancer[val.image_id[i]]

C:\Users\nguye\AppData\Local\Temp\ipykernel_18656\871094203.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val.cancer[i] = val.cancer[i] and dict_cancer[val.image_id[i]]


In [12]:
data = pd.concat([train, val, test])
data

,Unnamed: 0,patient_id,image_id,view,laterality,cancer
0,0,10130,1360338805,CC,L,1
1,1,10130,1672636630,MLO,L,1
2,2,10130,613462606,CC,L,1
3,3,10130,388811999,MLO,L,1
4,4,10226,530620473,CC,L,1
...,...,...,...,...,...,...
5119,5119,4607DDSM,5119,MLO,R,0
5120,5120,4608DDSM,5120,CC,L,0
5121,5121,4608DDSM,5121,MLO,L,0
5122,5122,4608DDSM,5122,CC,R,0


In [13]:
len(data.image_id.unique())

13430

In [34]:
cancer = data[data.cancer == 1].reset_index()
nocancer = data[data.cancer == 0].reset_index()

In [26]:
cancer

,index,Unnamed: 0,patient_id,image_id,view,laterality,cancer
0,0,0,10130,1360338805,CC,L,1
1,1,1,10130,1672636630,MLO,L,1
2,2,2,10130,613462606,CC,L,1
3,3,3,10130,388811999,MLO,L,1
4,4,4,10226,530620473,CC,L,1
...,...,...,...,...,...,...,...
6676,2711,2711,4193DDSM,2711,MLO,R,1
6677,2712,2712,4199DDSM,2712,CC,L,1
6678,2713,2713,4199DDSM,2713,MLO,L,1
6679,2714,2714,4199DDSM,2714,CC,R,1


In [15]:
len(cancer), len(nocancer)

(6681, 6783)

In [30]:
from remove_image_views import *

In [31]:
rv = move_image(cancer, 'Cancer')


In [38]:
rv.my_method(list(cancer.index))

In [37]:
for id in cancer.index[:1]:
    # print(id)
    par = cancer.patient_id[id]
    # print(sx)
    lat = cancer.laterality[id]
    view = cancer.view[id]
    file_name = f"D:/OneDrive - Industrial University of HoChiMinh City/WORKBASE/Project-rsna-breast-cancer-detection/Data_main/Dataset_VIEW/{par}/{view}/{lat}/{cancer.view[id]}_{cancer.laterality[id]}_{cancer.image_id[id]}.png"
    if os.path.exists(file_name) == False:
        print(par)

In [39]:
rv1 = move_image(nocancer, 'Nocancer')

rv1.my_method(list(nocancer.index))

In [36]:
cancer.patient_id[0]

'10130'

In [41]:
train.to_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\Rate11\train.csv')

In [42]:
val.to_csv(r'D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\Rate11\val.csv')

In [43]:
len(cancer)

6681

In [44]:
len(glob.glob('D:\OneDrive - Industrial University of HoChiMinh City\WORKBASE\Project-rsna-breast-cancer-detection\Data_main\CSV_MAIN\Rate11\Cancer\*.png'))

6681

In [ ]:
no_cancer_train, no_cancer_test_val = train_test_split(nocancer, test_size=0.3, random_state=42)
no_cancer_val, no_cancer_test = train_test_split(no_cancer_test_val,  test_size=0.3, random_state=42)